In [13]:
from pyspark.sql import SparkSession

# Spark entry point
spark = SparkSession \
    .builder \
    .appName("Aula Interativa 1 - Cientista de Dados - Apache Spark") \
    .getOrCreate()

spark.version

'3.5.1'

In [14]:
# Carregando a planilha CSV
titanic_df = spark.read.csv('./titanic-partial.csv',header='True',inferSchema='True')

# Detalhes dos atributos em https://www.kaggle.com/c/titanic/data?select=train.csv
titanic_df.printSchema()


root
 |-- PassengerId: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [15]:
# Carregando o arquivo JSON

survived_df = spark.read.json("./titanic-survived.json")
survived_df.printSchema()
survived_df.show()

root
 |-- PassengerId: long (nullable = true)
 |-- Survived: long (nullable = true)

+-----------+--------+
|PassengerId|Survived|
+-----------+--------+
|          1|       0|
|          2|       1|
|          3|       1|
|          4|       1|
|          5|       0|
|          6|       0|
|          7|       0|
|          8|       0|
|          9|       1|
|         10|       1|
|         11|       1|
|         12|       1|
|         13|       0|
|         14|       0|
|         15|       0|
|         16|       1|
|         17|       0|
|         18|       1|
|         19|       0|
|         20|       1|
+-----------+--------+
only showing top 20 rows



In [26]:
# Juntando os dois dataframes.

titanic_df = titanic_df.join(survived_df, ["PassengerId"])

titanic_df.printSchema()



root
 |-- PassengerId: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Survived: long (nullable = true)
 |-- Survived: long (nullable = true)



In [17]:
titanic_df.count()

891

In [18]:
titanic_df.show()

+-----------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+--------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Survived|
+-----------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+--------+
|          1|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|       0|
|          2|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|       1|
|          3|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|       1|
|          4|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|       1|
|          5|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|       0|
|          6|     3|    Moran, Mr. James|  male|NULL|    0|    0|          33087

In [19]:
titanic_df.select("Survived", "Name", "Pclass","Embarked").show()

+--------+--------------------+------+--------+
|Survived|                Name|Pclass|Embarked|
+--------+--------------------+------+--------+
|       0|Braund, Mr. Owen ...|     3|       S|
|       1|Cumings, Mrs. Joh...|     1|       C|
|       1|Heikkinen, Miss. ...|     3|       S|
|       1|Futrelle, Mrs. Ja...|     1|       S|
|       0|Allen, Mr. Willia...|     3|       S|
|       0|    Moran, Mr. James|     3|       Q|
|       0|McCarthy, Mr. Tim...|     1|       S|
|       0|Palsson, Master. ...|     3|       S|
|       1|Johnson, Mrs. Osc...|     3|       S|
|       1|Nasser, Mrs. Nich...|     2|       C|
|       1|Sandstrom, Miss. ...|     3|       S|
|       1|Bonnell, Miss. El...|     1|       S|
|       0|Saundercock, Mr. ...|     3|       S|
|       0|Andersson, Mr. An...|     3|       S|
|       0|Vestrom, Miss. Hu...|     3|       S|
|       1|Hewlett, Mrs. (Ma...|     2|       S|
|       0|Rice, Master. Eugene|     3|       Q|
|       1|Williams, Mr. Cha...|     2|  

In [21]:
titanic_df.describe().show()

+-------+-----------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+-------------------+
|summary|      PassengerId|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|           Survived|
+-------+-----------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+-------------------+
|  count|              891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|                891|
|   mean|            446.0| 2.308641975308642|                NULL|  NULL| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.2042079685746| NULL|    N

In [22]:
titanic_df.describe('Age', 'Fare').show()

+-------+------------------+-----------------+
|summary|               Age|             Fare|
+-------+------------------+-----------------+
|  count|               714|              891|
|   mean| 29.69911764705882| 32.2042079685746|
| stddev|14.526497332334035|49.69342859718089|
|    min|              0.42|              0.0|
|    max|              80.0|         512.3292|
+-------+------------------+-----------------+



In [23]:
titanic_df.select('Age', 'Fare').summary().show()

+-------+------------------+-----------------+
|summary|               Age|             Fare|
+-------+------------------+-----------------+
|  count|               714|              891|
|   mean| 29.69911764705882| 32.2042079685746|
| stddev|14.526497332334035|49.69342859718089|
|    min|              0.42|              0.0|
|    25%|              20.0|           7.8958|
|    50%|              28.0|          14.4542|
|    75%|              38.0|             31.0|
|    max|              80.0|         512.3292|
+-------+------------------+-----------------+



In [24]:
# narrow ou wide?
titanic_df.groupBy("Survived").count().show()


+--------+-----+
|Survived|count|
+--------+-----+
|       0|  549|
|       1|  342|
+--------+-----+



In [25]:
titanic_df.groupBy("Sex","Survived").count().show()

+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|  male|       0|  468|
|female|       1|  233|
|female|       0|   81|
|  male|       1|  109|
+------+--------+-----+



In [29]:
titanic_df.groupBy("Age").count().show()

+----+-----+
| Age|count|
+----+-----+
| 8.0|    4|
|70.0|    2|
| 7.0|    3|
|20.5|    1|
|49.0|    6|
|29.0|   20|
|40.5|    2|
|64.0|    2|
|47.0|    9|
|42.0|   13|
|24.5|    1|
|44.0|    9|
|35.0|   18|
|NULL|  177|
|62.0|    4|
|18.0|   26|
|80.0|    1|
|34.5|    1|
|39.0|   14|
| 1.0|    7|
+----+-----+
only showing top 20 rows



In [36]:
titanic_df.groupBy("Pclass","Survived").count().show()



AnalysisException: [AMBIGUOUS_REFERENCE] Reference `Survived` is ambiguous, could be: [`Survived`, `Survived`].

In [35]:
pandas_df = titanic_df.toPandas()

pandas_df.info()

ModuleNotFoundError: No module named 'distutils'

In [69]:
# https://insaid.medium.com/eda-with-pyspark-1f29b7d1618
# https://github.com/dvgodoy/handyspark
# Traz capacidades do Pàndas para o pySpark!
# Não funciona completamente no Spark 3 =/ https://github.com/dvgodoy/handyspark/issues/25
from handyspark import *

from matplotlib import pyplot as plt

fig, axs = plt.subplots(1, 2, figsize=(12, 4))

hdf = titanic_df.toHandy()

hdf.cols['Embarked'].hist(ax=axs[0])
hdf.cols[['Fare', 'Age']].scatterplot(ax=axs[1])

ModuleNotFoundError: No module named 'handyspark'

In [32]:
# EMBARKED
titanic_df.groupBy("Embarked").count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|    NULL|    2|
|       C|  168|
|       S|  644|
+--------+-----+



In [37]:
# narrow ou wide?
titanic_df = titanic_df.na.fill({"Embarked" : 'S'})

In [38]:
titanic_df.groupBy("Embarked").count().show()

CodeCache: size=131072Kb used=46173Kb max_used=46195Kb free=84898Kb
 bounds [0x00000001089e8000, 0x000000010b758000, 0x00000001109e8000]
 total_blobs=15482 nmethods=14508 adapters=885
 compilation: disabled (not enough contiguous free space left)
+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|       C|  168|
|       S|  646|
+--------+-----+



Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


In [39]:
# AGE
titanic_df.describe('Age').show()

+-------+------------------+
|summary|               Age|
+-------+------------------+
|  count|               714|
|   mean| 29.69911764705882|
| stddev|14.526497332334035|
|    min|              0.42|
|    max|              80.0|
+-------+------------------+



In [52]:
from pyspark.sql.functions import col, regexp_extract


titanic_df = titanic_df.withColumn("Saudacao", regexp_extract(col("Name"),"([A-Za-z]+)\.",1))

# Miss: Senhorita
# Mrs: Senhora
titanic_df.select("Saudacao", "Name").show()

<>:4: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
/var/folders/k9/z6_7b2f55tldshd60cfk17180000gn/T/ipykernel_19949/268891187.py:4: SyntaxWarning: invalid escape sequence '\.'
  titanic_df = titanic_df.withColumn("Saudacao", regexp_extract(col("Name"),"([A-Za-z]+)\.",1))


+--------+--------------------+
|Saudacao|                Name|
+--------+--------------------+
|      Mr|Braund, Mr. Owen ...|
|     Mrs|Cumings, Mrs. Joh...|
|    Miss|Heikkinen, Miss. ...|
|     Mrs|Futrelle, Mrs. Ja...|
|      Mr|Allen, Mr. Willia...|
|      Mr|    Moran, Mr. James|
|      Mr|McCarthy, Mr. Tim...|
|  Master|Palsson, Master. ...|
|     Mrs|Johnson, Mrs. Osc...|
|     Mrs|Nasser, Mrs. Nich...|
|    Miss|Sandstrom, Miss. ...|
|    Miss|Bonnell, Miss. El...|
|      Mr|Saundercock, Mr. ...|
|      Mr|Andersson, Mr. An...|
|    Miss|Vestrom, Miss. Hu...|
|     Mrs|Hewlett, Mrs. (Ma...|
|  Master|Rice, Master. Eugene|
|      Mr|Williams, Mr. Cha...|
|     Mrs|Vander Planke, Mr...|
|     Mrs|Masselmani, Mrs. ...|
+--------+--------------------+
only showing top 20 rows



In [45]:
titanic_df.select("Saudacao").distinct().show()


+--------+
|Saudacao|
+--------+
|     Don|
|    Miss|
|Countess|
|     Col|
|     Rev|
|    Lady|
|  Master|
|     Mme|
|    Capt|
|      Mr|
|      Dr|
|     Mrs|
|     Sir|
|Jonkheer|
|    Mlle|
|   Major|
|      Ms|
+--------+



In [70]:
titanic_df = titanic_df.replace(['Mlle','Mme', 'Ms', 'Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
               ['Miss','Miss','Miss','Mr','Mr',  'Mrs',  'Mrs',  'Other',  'Other','Other','Mr','Mr','Mr'])

In [71]:
titanic_df.select("Saudacao").distinct().show()

+--------+
|Saudacao|
+--------+
|    Miss|
|   Other|
|  Master|
|      Mr|
|     Mrs|
+--------+



In [68]:
titanic_df.groupby('Saudacao').avg('Age').collect()


[Row(Saudacao='Miss', avg(Age)=21.86),
 Row(Saudacao='Other', avg(Age)=45.888888888888886),
 Row(Saudacao='Master', avg(Age)=4.574166666666667),
 Row(Saudacao='Mr', avg(Age)=32.73960880195599),
 Row(Saudacao='Mrs', avg(Age)=35.981818181818184)]

In [50]:
from pyspark.sql.functions import col

titanic_df.filter(col("Saudacao") == 'Miss').select("Name", "Age", "Saudacao").show(5)

titanic_df.filter(titanic_df.Saudacao == 'Miss').select("Name", "Age", "Saudacao").show(5)

24/06/12 20:07:03 ERROR CodeGenerator: failed to compile: org.codehaus.commons.compiler.InternalCompilerException: Compiling "GeneratedClass" in File 'generated.java', Line 1, Column 1: File 'generated.java', Line 42, Column 16: Compiling "processNext()"
org.codehaus.commons.compiler.InternalCompilerException: Compiling "GeneratedClass" in File 'generated.java', Line 1, Column 1: File 'generated.java', Line 42, Column 16: Compiling "processNext()"
	at org.codehaus.janino.UnitCompiler.compile2(UnitCompiler.java:402)
	at org.codehaus.janino.UnitCompiler.access$000(UnitCompiler.java:236)
	at org.codehaus.janino.UnitCompiler$2.visitCompilationUnit(UnitCompiler.java:363)
	at org.codehaus.janino.UnitCompiler$2.visitCompilationUnit(UnitCompiler.java:361)
	at org.codehaus.janino.Java$CompilationUnit.accept(Java.java:371)
	at org.codehaus.janino.UnitCompiler.compileUnit(UnitCompiler.java:361)
	at org.codehaus.janino.SimpleCompiler.cook(SimpleCompiler.java:264)
	at org.codehaus.janino.ClassBodyE

+--------------------+----+--------+
|                Name| Age|Saudacao|
+--------------------+----+--------+
|Heikkinen, Miss. ...|26.0|    Miss|
|Sandstrom, Miss. ...| 4.0|    Miss|
|Bonnell, Miss. El...|58.0|    Miss|
|Vestrom, Miss. Hu...|14.0|    Miss|
|"McGowan, Miss. A...|15.0|    Miss|
+--------------------+----+--------+
only showing top 5 rows



24/06/12 20:07:06 ERROR CodeGenerator: failed to compile: org.codehaus.commons.compiler.InternalCompilerException: Compiling "GeneratedClass" in File 'generated.java', Line 1, Column 1: File 'generated.java', Line 42, Column 16: Compiling "processNext()"
org.codehaus.commons.compiler.InternalCompilerException: Compiling "GeneratedClass" in File 'generated.java', Line 1, Column 1: File 'generated.java', Line 42, Column 16: Compiling "processNext()"
	at org.codehaus.janino.UnitCompiler.compile2(UnitCompiler.java:402)
	at org.codehaus.janino.UnitCompiler.access$000(UnitCompiler.java:236)
	at org.codehaus.janino.UnitCompiler$2.visitCompilationUnit(UnitCompiler.java:363)
	at org.codehaus.janino.UnitCompiler$2.visitCompilationUnit(UnitCompiler.java:361)
	at org.codehaus.janino.Java$CompilationUnit.accept(Java.java:371)
	at org.codehaus.janino.UnitCompiler.compileUnit(UnitCompiler.java:361)
	at org.codehaus.janino.SimpleCompiler.cook(SimpleCompiler.java:264)
	at org.codehaus.janino.ClassBodyE

+--------------------+----+--------+
|                Name| Age|Saudacao|
+--------------------+----+--------+
|Heikkinen, Miss. ...|26.0|    Miss|
|Sandstrom, Miss. ...| 4.0|    Miss|
|Bonnell, Miss. El...|58.0|    Miss|
|Vestrom, Miss. Hu...|14.0|    Miss|
|"McGowan, Miss. A...|15.0|    Miss|
+--------------------+----+--------+
only showing top 5 rows



In [51]:
# Não tem mais idade faltante!
titanic_df.describe('Age').show()

+-------+------------------+
|summary|               Age|
+-------+------------------+
|  count|               714|
|   mean| 29.69911764705882|
| stddev|14.526497332334035|
|    min|              0.42|
|    max|              80.0|
+-------+------------------+



In [57]:
from pyspark.sql.functions import when

titanic_df = titanic_df.withColumn("Age",when((titanic_df["Saudacao"] == "Miss") & (titanic_df["Age"].isNull()), 22).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Saudacao"] == "Other") & (titanic_df["Age"].isNull()), 46).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Saudacao"] == "Master") & (titanic_df["Age"].isNull()), 5).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Saudacao"] == "Mr") & (titanic_df["Age"].isNull()), 33).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Saudacao"] == "Mrs") & (titanic_df["Age"].isNull()), 36).otherwise(titanic_df["Age"]))


In [54]:
titanic_df.filter(col("Saudacao") == 'Master').describe('Age').show()

24/06/12 20:08:34 ERROR CodeGenerator: failed to compile: org.codehaus.commons.compiler.InternalCompilerException: Compiling "GeneratedClass" in File 'generated.java', Line 1, Column 1: File 'generated.java', Line 2941, Column 16: Compiling "processNext()"
org.codehaus.commons.compiler.InternalCompilerException: Compiling "GeneratedClass" in File 'generated.java', Line 1, Column 1: File 'generated.java', Line 2941, Column 16: Compiling "processNext()"
	at org.codehaus.janino.UnitCompiler.compile2(UnitCompiler.java:402)
	at org.codehaus.janino.UnitCompiler.access$000(UnitCompiler.java:236)
	at org.codehaus.janino.UnitCompiler$2.visitCompilationUnit(UnitCompiler.java:363)
	at org.codehaus.janino.UnitCompiler$2.visitCompilationUnit(UnitCompiler.java:361)
	at org.codehaus.janino.Java$CompilationUnit.accept(Java.java:371)
	at org.codehaus.janino.UnitCompiler.compileUnit(UnitCompiler.java:361)
	at org.codehaus.janino.SimpleCompiler.cook(SimpleCompiler.java:264)
	at org.codehaus.janino.ClassB

+-------+------------------+
|summary|               Age|
+-------+------------------+
|  count|                36|
|   mean| 4.574166666666667|
| stddev|3.6198716433439615|
|    min|              0.42|
|    max|              12.0|
+-------+------------------+



In [60]:
titanic_df.describe("Cabin").show()

+-------+-----+
|summary|Cabin|
+-------+-----+
|  count|  204|
|   mean| null|
| stddev| null|
|    min|  A10|
|    max|    T|
+-------+-----+



In [72]:
titanic_df = titanic_df.drop("Cabin")

titanic_df.printSchema()


root
 |-- PassengerId: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = false)
 |-- Survived: long (nullable = true)
 |-- Survived: long (nullable = true)
 |-- Saudacao: string (nullable = true)
 |-- Alone: integer (nullable = false)
 |-- Family_Size: integer (nullable = true)



In [64]:
titanic_df = titanic_df.withColumn("Family_Size",col('SibSp')+col('Parch') + 1)

In [65]:
from pyspark.sql.functions import col


titanic_df.groupBy("Family_Size").count().show()


+-----------+-----+
|Family_Size|count|
+-----------+-----+
|          1|  537|
|          6|   22|
|          3|  102|
|          5|   15|
|          4|   29|
|          8|    6|
|          7|   12|
|         11|    7|
|          2|  161|
+-----------+-----+



In [73]:
from pyspark.sql.functions import asc

titanic_df.select("Name", "Family_Size").orderBy(col("Family_Size").desc()).show()


+--------------------+-----------+
|                Name|Family_Size|
+--------------------+-----------+
|Sage, Master. Tho...|         11|
|Sage, Miss. Const...|         11|
| Sage, Mr. Frederick|         11|
|Sage, Mr. George ...|         11|
|Sage, Miss. Stell...|         11|
|Sage, Mr. Douglas...|         11|
|"Sage, Miss. Doro...|         11|
|Goodwin, Master. ...|          8|
|Goodwin, Miss. Li...|          8|
|Goodwin, Master. ...|          8|
|Goodwin, Master. ...|          8|
|Goodwin, Mrs. Fre...|          8|
|Goodwin, Mr. Char...|          8|
|Andersson, Mr. An...|          7|
|Asplund, Mrs. Car...|          7|
|Andersson, Miss. ...|          7|
|Andersson, Miss. ...|          7|
|Asplund, Master. ...|          7|
|Asplund, Miss. Li...|          7|
|Asplund, Master. ...|          7|
+--------------------+-----------+
only showing top 20 rows



In [58]:
from pyspark.sql.functions import lit

titanic_df = titanic_df.withColumn('Alone',lit(0))


In [67]:
from pyspark.sql.functions import lit


titanic_df = titanic_df.withColumn('Alone',lit(0))
titanic_df = titanic_df.withColumn("Alone",when(titanic_df["Family_Size"] == 0, 1).otherwise(titanic_df["Alone"]))



NameError: name 'when' is not defined

In [ ]:
titanic_df.columns


In [ ]:
spark.conf.set("spark.sql.codegen.wholeStage", False)

titanic_df.filter(titanic_df.Age > 70).show()


In [ ]:
from pyspark.sql.functions import desc


titanic_df.orderBy(desc("age")).show(1)


In [ ]:
from pyspark.sql.functions import asc

titanic_df.orderBy(asc("age")).show(1)

In [67]:
titanic_df.groupBy('survived').agg({'age': 'avg'}).show()

+--------+-----------------+
|survived|         avg(age)|
+--------+-----------------+
|       0|30.80054644808743|
|       1|28.30312865497076|
+--------+-----------------+



In [68]:
titanic_df.stat.corr("age", "fare")


0.08908669546299611

In [69]:
titanic_df.stat.corr("age", "family_size")


-0.28226043774238513

In [70]:
# https://spark.apache.org/docs/latest/api/python//reference/pyspark.sql/api/pyspark.sql.DataFrameStatFunctions.html#pyspark.sql.DataFrameStatFunctions
titanic_df.stat.crosstab("Embarked", "PClass").show()

+---------------+---+---+---+
|Embarked_PClass|  1|  2|  3|
+---------------+---+---+---+
|              S|129|164|353|
|              Q|  2|  3| 72|
|              C| 85| 17| 66|
+---------------+---+---+---+



In [ ]:
titanic_df.groupBy('pclass').agg({'fare': 'avg'}).show()

In [ ]:
titanic_df.filter(titanic_df.Sex == 'female') \
    .groupBy('pclass')\
    .agg({'fare': 'avg'}).show()

In [71]:
# UDFs ajudam plugar funções complexas
# extrair o last Name

from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

def getLastName(str):
    return str.split(",")[0]

getLastNameUDF = udf(lambda z:getLastName(z),StringType())   

titanic_df = titanic_df.withColumn("lastName", getLastNameUDF(titanic_df.Name))

In [72]:
titanic_df.select("lastName").distinct().show()

+-------------+
|     lastName|
+-------------+
|     "Madigan|
|     Pavlovic|
|         Saad|
|      Palsson|
| Thorneycroft|
|     Meanwell|
|      Markoff|
|       Porter|
|     Harrison|
|     Bissette|
|        Hampe|
|Lemberopolous|
|      Ekstrom|
|       Boulos|
|       Bourke|
|     McGovern|
|     Kvillner|
|   Goldenberg|
|        Keefe|
|       Leader|
+-------------+
only showing top 20 rows



In [ ]:
titanic_df.groupBy("lastName").count().show()

In [ ]:
# extrair o last Name

from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

def getLastNameFirstCharacter(str):
    return str.split(",")[0][0]

getLastNameFirstCharacterUDF = udf(lambda z:getLastNameFirstCharacter(z),StringType())   

titanic_df.withColumn("lastNameFirstCharacter", getLastNameFirstCharacterUDF(titanic_df.Name)) \
    .groupBy("lastNameFirstCharacter") \
    .count().sort("count", ascending=False) \
    .show()